In [278]:
import pandas as pd
import ris
from ris import db2
from ris.db2 import pg_shp as shp
import os
from tqdm import tqdm
from IPython.display import clear_output
import getpass
import datetime
from IPython.display import Markdown
from dateutil.relativedelta import relativedelta
Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {}'.format(
    datetime.datetime.now().strftime('%Y-%m-%d %H:%M'), getpass.getuser(), ris.__version__
))

<strong>Notebook run on: 2019-07-11 16:42 | by bfeng | Using ris library version: 1.3.6

In [ ]:
db = db2.PostgresDb('dotdevpgsql02','CRASHDATA')

In [108]:
forms = db2.SqlDb('dot55sql01', 'forms', user='arcgis', db_pass='arcgis', quiet = True)

In [313]:
nodes = db2.query_to_table(db,'select nodeid from node where is_int = true').nodeid.tolist()

# Leap Years?

In [295]:
def five_crashes_rolling(nodes):
    end = datetime.datetime.now().strftime('%Y-%m-%d') 
    start = (datetime.datetime.now() - relativedelta(years=3)).strftime('%Y-%m-%d')
        
# Consider only nodes with 5+ crashes
    fiveplus = db2.query_to_table(forms,"""select nodeid, count(distinct integration_id) from forms.dbo.WC_ACCIDENT_F
    where year(accident_dt) >= 2017
    and accident_dt between '{start}' and '{end}'
    and nodeid is not null
    and nodeid in (""".format(start = start, end = end) + """,""".join(str(node) for node in nodes) + """)
    group by nodeid
    having count(distinct integration_id) >= 5""").nodeid.tolist()
    
    nodes = (set(nodes).intersection(set(fiveplus)))
    
# eliminate nodes with exactly 5 crashes, but over more than 365 day period
    too_long = db2.query_to_table(forms,"""select nodeid, count(distinct integration_id) from forms.dbo.WC_ACCIDENT_F
    where year(accident_dt) >= 2017
    and accident_dt between '{start}' and '{end}'
    and nodeid is not null
    and nodeid in (""".format(start = start, end = end) + """,""".join(str(node) for node in nodes) + """)
    group by nodeid
    having count(distinct integration_id) = 5 and datediff(day, min(accident_dt), max(accident_dt)) > 365
    """).nodeid.tolist()
    
    nodes = nodes - set(too_long)
    
# skip check for nodes with 5+ crashes, where earliest and latest crashes within a year of each other    
    verified = db2.query_to_table(forms,"""select nodeid, count(distinct integration_id) from forms.dbo.WC_ACCIDENT_F
    where year(accident_dt) >= 2017
    and accident_dt between '{start}' and '{end}'
    and nodeid is not null
    and nodeid in (""".format(start = start, end = end) + """,""".join(str(node) for node in nodes) + """)
    group by nodeid
    having count(distinct integration_id) >= 5 and datediff(day, min(accident_dt), max(accident_dt)) <= 365
    """).nodeid.tolist()
    
    nodes = nodes - set(verified)

# get crashes for 6+ crashes where max - min > 1 year
    crashes = db2.query_to_table(forms, """select nodeid, integration_id, cast(accident_dt as date) date from forms.dbo.WC_ACCIDENT_F 
    where nodeid in (""" + """,""".join(str(node) for node in nodes) + """)
    and year(accident_dt) >= 2017
    and accident_dt between '{start}' and '{end}'
    and nodeid is not null
    """.format(start = start, end = end))
    
    crashes = crashes.sort_values(by=['nodeid','date'])
    crashes['date'] = pd.to_datetime(crashes['date']) # convert text to date
    crashes['dif'] = crashes.groupby('nodeid').date.diff(periods=4) # take every 5th date difference per node
    crashes['dif'] = crashes['dif'].dt.days # convert to int
    crashes['five_yr'] = 0
    crashes.loc[crashes.dif <= 365, 'five_yr'] = 1 # boolean for 5 crashes within 365 days
    passed = pd.DataFrame(crashes.groupby('nodeid').five_yr.sum()) # sum booleans by nodeid
    passed = passed.loc[crashes2.five_yr > 0] # only take nodeids with at least one 5 crash period in 365 days
    crashes2 = crashes2.reset_index()
    nodes_passed = crashes2.nodeid.tolist()
    
# grab crashes from nodes with 5+ crashes, where earliest and latest crashes within a year of each other 
# immediately passes condition 
    verified_crashes = db2.query_to_table(forms, """select nodeid, integration_id, cast(accident_dt as date) date from forms.dbo.WC_ACCIDENT_F 
    where nodeid in (""" + """,""".join(str(node) for node in verified) + """)
    and year(accident_dt) >= 2017
    and accident_dt between '{start}' and '{end}'
    and nodeid is not null
    """.format(start = start, end = end))
    
    crashes = crashes.loc[crashes.node.isin(nodes_passed)]
    crashes = pd.concat([crashes[verified_crashes.columns.tolist()], verified_crashes])
    
    return crashes